In [30]:
with open("Grammar.txt", "r", encoding='utf-8') as f:
    rows = f.readlines()

# print(rows[0])
# print(rows[1])

In [24]:
rows[2]

'СоставнойОператор ::= Оператор СоставнойОператор\n'

In [22]:
rows[3]

'СоставнойОператор [/eps] ::= /eps\n'

In [32]:
new_rows = []
for i, row in enumerate(rows):
    words = row.split(' ')
    if words[0] != '\n':
        new_row = str(i + 1) + ' ' + words[0]
        new_row += ' '.join(words[1:])
        new_rows.append(new_row)

print(new_rows)
len(new_rows)
# new_rows2 = []
# for i, row in enumerate(new_rows):
    

['1 S::= СписокОператоров\n', '3 ОбъявлениеСтруктуры[int] ::= ТипСтруктуры ID Структура\n', '4 ОбъявлениеСтруктуры[bool] ::= ТипСтруктуры ID Структура\n', '5 ОбъявлениеСтруктуры[void] ::= ТипСтруктуры ID Структура\n', '7 ТипСтруктуры[int] ::= int\n', '8 ТипСтруктуры[void] ::= void\n', '9 ТипСтруктуры[bool] ::= bool\n', '11 Структура[(] ::= ( ПараметрыФункции) СоставнойОператор\n', '12 Структура[=] ::= = Выражение\n', '13 Структура[/eps] ::= /eps\n', '15 ПараметрыФункции[int bool void] ::= ПараметрФункции СписокПараметровФункции\n', '16 ПараметрыФункции[/eps] ::= /eps\n', '17 ПараметрФункции[int bool void] ::= ТипСтруктуры ID ИнициализацияПеременной , СписокПараметровФункции\n', '18 ПараметрФункции[/eps] ::= /eps\n', '20 СписокПараметровФункции[,] ::= , ПараметрыФункции\n', '22 ИнициализацияПеременной[=] ::= = Выражение\n', '23 ИнициализацияПеременной[/eps] ::= /eps\n', '25 СоставнойОператор[{] ::= { СписокОператоров }\n', '26 СписокОператоров[if while return ID int bool void NUM true f

69